In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import files
uploaded = files.upload()
df = pd.read_csv("tennis_article.csv")
df.head()

Saving tennis_article.csv to tennis_article (1).csv


,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [ ]:
df['article_text'][0]

"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the same 

In [ ]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))
 
sentences = [y for x in sentences for y in x]
sentences[0]

'Maria Sharapova has basically no friends as tennis players on the WTA Tour.'

In [ ]:
from google.colab import files
#uploaded = files.upload()

In [ ]:
word_embeddings = {}
nltk.download('stopwords')
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
 
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences[1]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


'russian player problems openly speaking recent interview said really hide feelings much'

In [ ]:
clean_sentences[0]

'maria sharapova basically friends tennis players wta tour'

In [ ]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)
  

In [ ]:
sentence_vectors[1]

array([-8.46056044e-02,  1.69623360e-01,  4.33749944e-01, -8.58418122e-02,
       -1.70975417e-01,  1.32896423e-01, -2.54114658e-01, -1.70107469e-01,
        2.79757172e-01,  6.33887276e-02, -2.04101473e-01,  7.64638111e-02,
        2.85677612e-01, -9.00602564e-02, -1.14498213e-01, -9.90290865e-02,
       -1.34927511e-01, -2.71166232e-03, -4.13565874e-01,  6.32358193e-02,
        4.55082878e-02,  1.91257671e-01, -1.39448941e-01, -1.13781996e-03,
       -9.70350802e-02, -1.26338288e-01, -5.15491217e-02, -3.08907747e-01,
        3.43927354e-01,  2.86936820e-01, -2.22957656e-01,  4.46837395e-01,
       -1.11253977e-01, -7.21814632e-02,  1.37357712e-01,  1.71515450e-01,
        5.39051816e-02,  2.76359648e-01, -2.55537611e-02, -1.31975263e-02,
       -4.84045923e-01, -1.34829834e-01,  2.57399052e-01, -1.33780092e-01,
       -2.33958840e-01, -1.68721929e-01,  2.29427516e-01, -3.33674073e-01,
       -2.39330102e-02, -7.64521301e-01,  2.71977514e-01, -9.60583910e-02,
        1.57525614e-01,  

In [ ]:
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
      

In [ ]:
sim_mat[1][5]

0.8124060034751892

In [ ]:
import networkx as nx
 
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
print(nx_graph)

Graph with 119 nodes and 6903 edges


In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(5):
  print("ARTICLE:",i)
  print(df['article_text'][i])
  print('\n')
  print("SUMMARY:",i)
  print(ranked_sentences[i][1])
  print('\n')

ARTICLE: 0
Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in